<a href="https://colab.research.google.com/github/kanyakornvee/CWT/blob/main/Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pystan==2.19.1.1
!pip uninstall fbprophet
!pip install cmdstanpy==0.9.68
!pip install prophet

Found existing installation: fbprophet 0.7.1
Uninstalling fbprophet-0.7.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/fbprophet-0.7.1-py3.7.egg-info
    /usr/local/lib/python3.7/dist-packages/fbprophet/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/fbprophet/stan_model/prophet_model.pkl
Proceed (y/n)? y
  Successfully uninstalled fbprophet-0.7.1


In [ ]:
import pandas as pd
import sklearn as sk
from prophet import Prophet

In [ ]:
df = pd.read_csv('/content/CWT/provider_level_data.csv')
df.head()
df.rename({'period': 'ds', 'performance': 'y'}, axis=1, inplace=True)
df['y'] = round(df['y']*100, 2)

In [ ]:
df = df[df.cancer_type == "ALL CANCERS"]
df.region_name.fillna("Unknown")
df['region_name'] = df['region_name'].astype(object)
df['standard'] = df['standard'].astype(object)
df = df.drop(["provider_name", "provider_code"], axis=1)
df['ds']= pd.to_datetime(df['ds'])
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ds,region_name,standard,cancer_type,total_treated,within_standard,breaches,y
3324,2016-01-01,NaN,31 Days,ALL CANCERS,3.0,3.0,0.0,100.00
3326,2016-01-01,NaN,31 Days,ALL CANCERS,31.0,24.0,7.0,77.42
3327,2016-01-01,NaN,31 Days,ALL CANCERS,1.0,0.0,1.0,0.00
3328,2016-01-01,NaN,31 Days,ALL CANCERS,2.0,2.0,0.0,100.00
3332,2016-01-01,NaN,31 Days,ALL CANCERS,5.0,5.0,0.0,100.00


In [ ]:
df['ds']. value_counts()

2017-11-01    2357
2017-08-01    2334
2017-03-01    2322
2018-03-01    2310
2017-10-01    2309
              ... 
2020-09-01     993
2020-05-01     980
2020-07-01     957
2020-08-01     955
2020-06-01     952
Name: ds, Length: 71, dtype: int64

In [ ]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = m.make_future_dataframe(periods=200)
future.tail()


,ds
224,2018-11-13
225,2018-11-14
226,2018-11-15
227,2018-11-16
228,2018-11-17
